### **title of whatever we're doing**

Amia Yi, Kevin Fan, Vincent ???

**Introduction**
- why are we analyzing this data and stuff
- introduce tutorial and what its for and why its useful
- look at the examples on the project desc for reference

In [1]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
df = pd.read_csv("C:\\Users\\aegir\\Documents\\dATASCI\\CMSC320\\Final Project\\Data\\3680211.csv")
df

,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION,DATE,CDSD,CDSD_ATTRIBUTES,CLDD,CLDD_ATTRIBUTES,...,PRCP,PRCP_ATTRIBUTES,SNOW,SNOW_ATTRIBUTES,TAVG,TAVG_ATTRIBUTES,TMAX,TMAX_ATTRIBUTES,TMIN,TMIN_ATTRIBUTES
0,USC00181995,"COLLEGE PARK, MD US",38.98333,-76.95,27.4,1895,NaN,NaN,NaN,NaN,...,877.3,",6",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,USC00181995,"COLLEGE PARK, MD US",38.98333,-76.95,27.4,1896,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,USC00181995,"COLLEGE PARK, MD US",38.98333,-76.95,27.4,1898,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,13.8,6.0,19.3,6.0,8.3,6.0
3,USC00181995,"COLLEGE PARK, MD US",38.98333,-76.95,27.4,1900,877.4,NaN,877.4,6.0,...,NaN,NaN,NaN,NaN,13.1,6.0,19.3,6.0,6.9,6.0
4,USC00181995,"COLLEGE PARK, MD US",38.98333,-76.95,27.4,1901,NaN,NaN,NaN,NaN,...,1239.3,",6",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,USC00180706,"BELTSVILLE PLANT STATION, MD US",39.01667,-76.95,67.1,1956,567.6,NaN,567.6,0.0,...,845.8,",0",280.0,",0",12.6,0.0,18.4,0.0,6.8,0.0
211,USC00180706,"BELTSVILLE PLANT STATION, MD US",39.01667,-76.95,67.1,1957,695.7,NaN,695.7,0.0,...,945.3,",0",945.0,",0",13.1,0.0,18.8,0.0,7.3,0.0
212,USC00180706,"BELTSVILLE PLANT STATION, MD US",39.01667,-76.95,67.1,1958,NaN,NaN,NaN,NaN,...,1098.2,",0",582.0,",0",NaN,NaN,NaN,NaN,NaN,NaN
213,USC00180706,"BELTSVILLE PLANT STATION, MD US",39.01667,-76.95,67.1,1959,810.8,NaN,810.8,0.0,...,895.1,",0",293.0,",0",13.4,0.0,19.5,0.0,7.3,0.0
